# Getting started with yieldplotlib

## Introduction
This tutorial will walk you thorugh some of the basic functionality of yieldplotlib to get you started on generating your own yield visualizations. To run yieldplotlib you need to have at least one AYO output folder, EXOSIMS output folder, or yield input package (YIP). Sample data has been provided in the tutorials folder of this repository for demonstrative purposes. 

You can also run series of pre-made plots for yield outputs which can be found in the yieldplotlib/src/scripts folder, or accessed through the yieldplotlib pipeline and command line interface.

### Imports

In [ ]:
import logging

import matplotlib.pyplot as plt
from IPython.display import HTML

import yieldplotlib as ypl
from yieldplotlib.accessibility import AccessibilityManager
from yieldplotlib.logger import logger
from yieldplotlib.plots.yip_plots import make_offax_psf_movie, plot_core_throughtput

### Loading Yield Data

 Yieldplotlib uses a key map to equate parameters between EXOSIMS and AYO (and
  potential future yield codes). This key map is pulled daily from a collaborative Google Sheet to the GitHub repository in the form of a key_map.csv file. To see all the possible named parameters that can be plotted in yieldplotlib, consult documentation or the first column of that CSV file/Google Sheet. 

In order to convert this CSV to the key_map.py file that is needed by yieldplotlib to parse the yield output directories, the user should run:

`python generate_key_map.py --csv key_map.csv`

This will populate a key_map.py file containing all the proper mappings. Now we can load our sample AYO and EXOSIMS data using the AYODirectory and EXOSIMSDirectory classes.  

In [ ]:
%%capture
logger.setLevel(logging.ERROR)
# Load the sample data provided in the GitHub repository
ayo = ypl.fetch_ayo_data()
exosims = ypl.fetch_exosims_data()


# Normally this looks more like this:
# ayo_folder = Path("sample_data/ayo")
# exosims_folder = Path("sample_data/exosims")
# ayo = AYODirectory(ayo_folder)
# exosims = EXOSIMSDirectory(exosims_folder)

Now that we have loaded our directories, we can display the file structure to see what is contained in our yield outputs at a glance.

In [ ]:
logger.setLevel(logging.WARNING)
print(exosims.display_tree())
print(ayo.display_tree())

## Accessing Data and Generating Plots 
We have our data loaded and can now start accessing it and generating 
plots!

### Get Data
Data can be accessed from the AYODirectory and EXOSIMSDirectory using the .get() function which takes the named yieldplotlib keys as input and outputs the corresponding values. This can be in the form of singular values or arrays.

In [ ]:
# Pupil diameter
ayo_pup = ayo.get("pupil_diam")
exo_pup = exosims.get("pupil_diam")
print(f"Pupil diameters are:\nAYO: {ayo_pup} \nEXOSIMS: {exo_pup}")

# ExoEarth yields
ayo_yield = ayo.get("yield_earth")
exo_yield = exosims.get("yield_earth")
print(f"\nExoEarth yields are:\nAYO: {ayo_yield} \nEXOSIMS: {exo_yield}")

# Target star distances (first 10)
ayo_dist = ayo.get("star_dist")
exo_dist = exosims.get("star_dist")
print(f"\nStellar distances (pc) are:\nAYO: {ayo_dist[:10]} \nEXOSIMS: {exo_dist[:10]}")

### Generic Plots

yieldplotlib expands directly on matplotlib to utilize its functionality and to make plot customization intuitive for those who have familiarity with matplotlib and its features. Below are examples of some of the generic yieldplotlib plots that one can make.

In [ ]:
# Histograms

fig, ax = plt.subplots(figsize=(8, 6))
ax.ypl_hist(exosims, x="star_dist", bins=20, alpha=0.7, label="EXOSIMS Stars")
ax.ypl_hist(ayo, x="star_dist", bins=20, alpha=0.7, label="AYO Stars")
ax.set_title("Distribution of Stellar Distances")
plt.show()

In [ ]:
# Scatter Plots
fig, ax = plt.subplots()
data = ax.ypl_scatter(ayo, x="star_dist", y="star_L", c="star_comp_det")
ax.set_title("HZ Completeness (AYO)")
ax.set_yscale("log")
plt.colorbar(data)

fig, ax = plt.subplots()
data = ax.ypl_scatter(exosims, x="star_dist", y="star_L", c="star_comp_det")
ax.set_title("HZ Completeness (EXOSIMS)")
ax.set_yscale("log")
plt.colorbar(data)
plt.show()

### Comparative Plots 

Instead of plotting datasets individually, for comparison purposes it is often desired to plot different yield outputs in a single figure. Here the suite of available yieldplotlib comparative plots should be used. Lets see two examples plotting the same HZ completeness plot as above. 

In [ ]:
# Two yield runs on same set of axes.

fig, ax = plt.subplots()
ax.set_title("HZ Completeness")
data = ypl.compare(
    ax,
    [exosims, ayo],
    x="star_dist",
    y="star_L",
    plot_type="scatter",
    c="star_comp_det",
)
ax.set_yscale("log")
plt.show()

In [ ]:
# Two yield runs on different sets of axes.

fig, axs = ypl.multi(
    [exosims, ayo],
    x="star_dist",
    y="star_L",
    plot_type="scatter",
    c="star_comp_det",
    sharex=True,
    sharey=True,
    titles=["EXOSIMS", "AYO"],
)
axs[0, 0].set_yscale("log")
plt.show()

Additionally, we can run more complex analyses on multi-dimensional grids of parameter space. 

In [ ]:
# Plot a grid of parameters against each other.
fig, axs = ypl.xy_grid(
    [ayo],
    ["star_L", "star_dist"],
    ["MV", "Ms"],
    plot_type="scatter",
    legend=True,
)
plt.show()

In [ ]:
# Plot multiple panels with different specifications for each yield run.

spec1 = {
    "x": "star_L",
    "y": "star_dist",
    "plot_type": "scatter",
    "c": "star_comp_det",
    "alpha": 0.7,
}
spec2 = {
    "x": "star_L",
    "y": "star_dist",
    "plot_type": "scatter",
    "c": "exp_time_char",
    "alpha": 0.7,
}
specs = [spec1, spec2]

fig, axes = ypl.panel(
    [exosims, ayo],
    *specs,
    figsize=(10, 10),
    suptitle=None,
    layout=None,
    sharex=True,
    sharey=True,
    titles=["HZ Completeness", "Exp Time Char", "", ""],
)
axes[1, 0].set_xscale("log")
plt.show()

## Yield Input Package (YIP) Loading
yieldplotlib supports the loading and plotting of yield input packages using yippy as a back end. YIPs specify relevant coronagraph parameters for yield codes and so are often critical inputs to be able to vet and visualize.  

In [ ]:
# Load the YIP data
yip = ypl.fetch_yip_data()
# Normally looks like this
# yip_folder = Path("sample_data/yip")
# yip = YIPDirectory(yip_folder)

In [ ]:
plot_core_throughtput(
    [exosims], ["EXOSIMS"], yip=yip, ax_kwargs={"xlim": (0, 32), "ylim": (0, 0.4)}
)

In [ ]:
# Make GIF animation of the off-axis stellar PSF.
ani = make_offax_psf_movie(yip, "offax_psf_animation.gif")

# Code to easily view the animation in the notebook.
HTML(ani.to_jshtml())

## Checking Accessibility 

yieldplotlib has a built-in class to check the accessibility of plots. This includes ensuring that colorbars are monotonic, have adequate range, and that all font sizes are large enough to be easily readable. This class is 
currently very basic and so is not designed to catch all potential accessibility issues. Instead, it is meant to alert for commonly encountered problems and to make the user think about their choices with intentionality.   

In [ ]:
# Passing Example
# Note: colored scatter points will always fail the broad monotonic color check.
plt.set_cmap("plasma")
fig, ax = plt.subplots()
data = ax.ypl_scatter(ayo, x="star_dist", y="star_L", c="star_comp_det")
ax.set_title("HZ Completeness (AYO)")
plt.colorbar(data)
plt.show()

am = AccessibilityManager(ax)
am.run_checks()

In [ ]:
# Failing Example
fig, ax = plt.subplots()

# Set a bad (non monotonic) colormap.
plt.set_cmap("gist_rainbow")

data = ax.ypl_scatter(ayo, x="star_dist", y="star_L", c="star_comp_det")
ax.set_title("HZ Completeness (AYO)")
plt.colorbar(data)

# Decrease Font Sizes
ax.set_xlabel("Star Dist", fontsize=6)
ax.tick_params(axis="both", which="major", labelsize=8)

ax.set_title("Distribution of Stellar Distances")
plt.show()

am = AccessibilityManager(ax)
am.run_checks()